In [1]:
import math

from scipy.stats import norm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import datetime
import scipy.stats as st
import scipy.optimize as optimize
import enum
import OptionLib as opt
%load_ext autoreload
%autoreload 2

In [2]:
strike = 190
option_type = 'call'
r = 0.075
exp_date = pd.to_datetime('2023-05-03')

In [3]:
opt_prices = pd.read_csv('data/SR190CE3A.csv')
opt_prices = opt_prices.set_index('date')
opt_prices.index = pd.to_datetime(opt_prices.index)

In [4]:
opt_prices['daysToExp'] = [(exp_date-pd.to_datetime(d)).days for d in list(opt_prices.index)]#np.full((550, ), 549) - np.arange(0, 550)

In [5]:
opt_prices

,price,underlying,daysToExp
date,,,
2023-04-11,30.29,218.60,22
2023-04-12,34.66,219.22,21
2023-04-13,32.91,219.85,20
2023-04-14,33.21,221.87,19
2023-04-17,37.64,228.10,16
2023-04-18,42.86,232.66,15
2023-04-19,44.74,232.95,14
2023-04-20,46.40,236.28,13
2023-04-21,47.08,235.17,12


In [25]:
opt_prices['bsm_iv'] = opt_prices.apply(lambda x:
                        opt.opt_vol(market_price=x[0],
                                    spot=x[1],
                                    strike=strike,
                                    r=r,
                                    time=x[2]/252, # with real data have column days to expiration and substitute here
                                    type=option_type),
                             axis=1)

/Users/artemilin/PycharmProjects/pythonProject/options/OptionLib.py:89: RuntimeWarning: divide by zero encountered in double_scalars
  return (np.log(spot / strike) + (r + sigma ** 2 / 2) * time) / (sigma * np.sqrt(time))


In [ ]:
opt_prices

In [41]:
opt_prices.head(10)

,price,underlying,daysToExp,bsm_iv,heston_iv
date,,,,,
2023-04-11,30.29,218.60,22,0.3130,0.313027
2023-04-12,34.66,219.22,21,0.6288,0.628852
2023-04-13,32.91,219.85,20,0.4905,0.490480
2023-04-14,33.21,221.87,19,0.3269,0.000100
2023-04-17,37.64,228.10,16,0.0914,0.000100
2023-04-18,42.86,232.66,15,0.1042,0.000100
2023-04-19,44.74,232.95,14,0.6105,0.610498
2023-04-20,46.40,236.28,13,0.1199,0.000100
2023-04-21,47.08,235.17,12,0.7151,0.715137


In [21]:
opt.BSM(r=0.075, spot=218.0, strike=190, time=22/252, sigma=0.0467, type='call')

29.23998371856584

In [ ]:
print(opt.heston_implied_vol('call', 5.57, 100, 100, 0.05, 1, 0))
def heston_implied_vol(option_type, option_price, S, K, r, T, q, h_params):
Heston(S0=S, strike=K, time=T, r=r, kappa=0.0,
                               theta=0.01,
                               v0=mid_vol**2,
                               lamda=0.0, sigma=0.000001, rho=-0.7)

S0=S,
strike=K
time=T
#r=r
kappa=0.0
theta=0.01,
v0=mid_vol**2
lamda=0.0
sigma=0.000001
rho=-0.7

In [22]:
h_params = {
    "S0": 218.60,
"strike": 190,
"r": 0.075,
"kappa": 0.0,
"theta": 0.01,

"lamda": 0.0,
"sigma": 0.000001,
"rho": 0.7
}



S0 = 218.60
strike = 190
time = 22/252
r = 0.075

kappa = h_params['kappa']
theta = h_params['theta']
v0 = 0.0467 # 0.01
lamda = h_params['lamda']
sigma = h_params['sigma']
rho = h_params['rho']

In [23]:
heston_model = opt.Heston(S0=S0, strike=strike, time=time, r=r, kappa=kappa, theta=theta, v0=v0, lamda=lamda, sigma=sigma, rho=rho)
heston_call = heston_model.price(0.00001, 100, 10000)['call']
heston_call

29.88763707481118

In [32]:
opt.heston_implied_vol(option_type, option_price=30.29, S=218.60, K=190, r=0.075, T=22/252, q=1, h_params=h_params)

0.31302673944830894

In [33]:
opt_prices['heston_iv'] = opt_prices.apply(lambda x:
                                           opt.heston_implied_vol(option_type='call',
                                            option_price=x[0], S=x[1], K=strike, r=r, T=x[2]/252, q=1, h_params=h_params),
                                            axis=1)

In [35]:
opt_prices.to_csv('data/sber_ivs.csv')

In [ ]:
We do alibrate sabr params so it would match iv of market so in this case we can juct adress market's iv